In [1]:
import duckdb
import os

In [2]:
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
os.getenv('AWS_ACCESS_KEY_ID')

'AKIAD58EA06099245AB3'

In [4]:
try:
    # Delete the secret if it exists
    duckdb.sql("DROP SECRET IF EXISTS secret1")
except Exception as e:
    print(e)

duckdb.sql(
    f"""
CREATE SECRET secret1 (
    TYPE S3,
    KEY_ID '{os.getenv('AWS_ACCESS_KEY_ID')}',
    SECRET '{os.getenv('AWS_SECRET_ACCESS_KEY')}',
    ENDPOINT '{os.getenv('AWS_S3_ENDPOINT')}'
);
    """
)

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [5]:
duckdb.sql(
    """
CREATE OR REPLACE TABLE weather_forecast AS
    SELECT
        *
    FROM 's3://md-raw-data/*/forecast_lv_grid.parquet'
    """
)


In [6]:
duckdb.sql(
    """
CREATE OR REPLACE TABLE weather_current AS
    SELECT
        *
    FROM 's3://md-raw-data/*/current_weather_lv_grid.parquet'
    """
)


In [8]:
display(duckdb.sql("SELECT * FROM weather_forecast WHERE iso_3166_2 = 'LV-056' AND model = 'meteofrance_seamless' AND date = '2025-01-26T00:00:00+00:00'"))
duckdb.sql("SELECT * FROM weather_current WHERE iso_3166_2 = 'LV-056' AND model = 'meteofrance_seamless' AND date = '2025-01-26T00:00:00+00:00'")

┌──────────────────────────┬──────────────────────────┬─────────┬──────────┬───────────┬───────────┬─────────────┬────────────┬────────────────┬───────────────────┬───────────┬───────────────┬───────────────────────────┬──────────────┬─────────────┬─────────────────┬─────────────────┬──────────────────┬────────────┐
│        created_at        │           date           │  model  │ latitude │ longitude │ elevation │ temperature │ wind_speed │ wind_direction │ relative_humidity │ dew_point │ precipitation │ precipitation_probability │ weather_code │ cloud_cover │ cloud_cover_low │ cloud_cover_mid │ cloud_cover_high │ iso_3166_2 │
│ timestamp with time zone │ timestamp with time zone │ varchar │  double  │  double   │  double   │    float    │   float    │     float      │       float       │   float   │     float     │           float           │    float     │    float    │      float      │      float      │      float       │  varchar   │
├──────────────────────────┴──────────────────

┌──────────────────────────┬──────────────────────────┬─────────┬──────────┬───────────┬───────────┬─────────────┬────────────┬────────────────┬───────────────────┬───────────┬───────────────┬───────────────────────────┬──────────────┬─────────────┬─────────────────┬─────────────────┬──────────────────┬────────────┐
│        created_at        │           date           │  model  │ latitude │ longitude │ elevation │ temperature │ wind_speed │ wind_direction │ relative_humidity │ dew_point │ precipitation │ precipitation_probability │ weather_code │ cloud_cover │ cloud_cover_low │ cloud_cover_mid │ cloud_cover_high │ iso_3166_2 │
│ timestamp with time zone │ timestamp with time zone │ varchar │  double  │  double   │  double   │   double    │   double   │     double     │      double       │  double   │    double     │          double           │    double    │   double    │     double      │     double      │      double      │  varchar   │
├──────────────────────────┴──────────────────

In [19]:
weather_variables = [
        "temperature",
        # "wind_speed",
        # "wind_direction",
        # "relative_humidity",
        # "dew_point",
        # "apparent_temperature",
        # "precipitation",
        # "precipitation_probability",
        # "rain",
        # "showers",
        # "snowfall",
        # "snow_depth",
        # "weather_code",
        # "cloud_cover",
        # "cloud_cover_low",
        # "cloud_cover_mid",
        # "cloud_cover_high",
]

diff_statements = [
    f"abs(forecast_lv_grid.{variable} - current_weather_lv_grid.{variable}) AS {variable}_difference"
    for variable in weather_variables
]


In [32]:
duckdb.sql(
    f"""
SELECT
    forecast_lv_grid.date,
    forecast_lv_grid.model,
    current_weather_lv_grid.model,
    forecast_lv_grid.latitude,
    forecast_lv_grid.longitude,
    forecast_lv_grid.date - forecast_lv_grid.created_at AS forecast_lead_time,
    {", ".join(diff_statements)},
    forecast_lv_grid.temperature,
    current_weather_lv_grid.temperature,
    forecast_lv_grid.iso_3166_2,
    current_weather_lv_grid.iso_3166_2
FROM forecast_lv_grid
JOIN current_weather_lv_grid ON
    forecast_lv_grid.date = current_weather_lv_grid.date
    AND forecast_lv_grid.latitude = current_weather_lv_grid.latitude
    AND forecast_lv_grid.longitude = current_weather_lv_grid.longitude
WHERE forecast_lv_grid.iso_3166_2 = 'LV-056'
    -- AND abs(forecast_lv_grid.temperature - current_weather_lv_grid.temperature) > 10
    AND forecast_lv_grid.date = '2025-01-26T00:00:00+00:00'
    AND forecast_lv_grid.model = 'ukmo_seamless'
    """
)


┌──────────────────────────┬───────────────┬──────────────────────┬──────────┬───────────┬────────────────────┬────────────────────────┬─────────────┬────────────────────┬────────────┬────────────┐
│           date           │     model     │        model         │ latitude │ longitude │ forecast_lead_time │ temperature_difference │ temperature │    temperature     │ iso_3166_2 │ iso_3166_2 │
│ timestamp with time zone │    varchar    │       varchar        │  double  │  double   │      interval      │         double         │    float    │       double       │  varchar   │  varchar   │
├──────────────────────────┼───────────────┼──────────────────────┼──────────┼───────────┼────────────────────┼────────────────────────┼─────────────┼────────────────────┼────────────┼────────────┤
│ 2025-01-26 02:00:00+02   │ ukmo_seamless │ ukmo_seamless        │    26.25 │     56.25 │ -12:08:22.169663   │    0.34999942779541016 │         8.4 │  8.050000190734863 │ LV-056     │ LV-056     │
│ 2025-01-

In [31]:
duckdb.sql("SELECT * FROM current_weather_lv_grid WHERE iso_3166_2 = 'LV-056' AND temperature > 19")

┌───────────────────────────────┬──────────────────────────┬──────────────────────┬────────────────────┬───────────────────┬───────────┬────────────────────┬─────────────────────┬────────────────────┬───────────────────┬────────────────────┬───────────────┬───────────────────────────┬──────────────┬─────────────┬─────────────────┬───────────────────┬──────────────────┬────────────┐
│          created_at           │           date           │        model         │      latitude      │     longitude     │ elevation │    temperature     │     wind_speed      │   wind_direction   │ relative_humidity │     dew_point      │ precipitation │ precipitation_probability │ weather_code │ cloud_cover │ cloud_cover_low │  cloud_cover_mid  │ cloud_cover_high │ iso_3166_2 │
│   timestamp with time zone    │ timestamp with time zone │       varchar        │       double       │      double       │  double   │       double       │       double        │       double       │      double       │       dou

In [36]:
duckdb.sql(
    """
SELECT iso_3166_2 AS iso_3166_2, AVG(temperature_performance) AS "AVG(temperature_performance)" 
FROM (SELECT * FROM 's3://processed-data/*/*.parquet'
) AS virtual_table WHERE iso_3166_2 = 'LV-056' GROUP BY iso_3166_2 
 LIMIT 50000;
    """
)

┌────────────┬──────────────────────────────┐
│ iso_3166_2 │ AVG(temperature_performance) │
│  varchar   │            double            │
├────────────┼──────────────────────────────┤
│ LV-056     │            780.1897159502973 │
└────────────┴──────────────────────────────┘